<a href="https://colab.research.google.com/github/aurora995/tesi-watermarking-gcolab/blob/main/pytorch_Resnet32_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn.functional as F
import torch
from torch import nn
from torchvision import models
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

from google.colab import files

from torch.optim.lr_scheduler import *

import sys 
import seaborn as sns

import glob
import os

gdrive_location = '/content/drive/MyDrive/TESI/Saved_Data/'

RESNET CIFAR10

In [2]:
class ResnetLambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(ResnetLambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class ResnetBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(ResnetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = None
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                Increses dimension via padding, performs identity operations
                """
                self.shortcut = ResnetLambdaLayer(lambda x:
                                                  F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes // 4, planes // 4),
                                                        "constant",
                                                        0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(self.expansion * planes)
                )

        self.relu2 = nn.ReLU(inplace=False)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.shortcut is not None:
            identity = self.shortcut(x)

        out += identity
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, option="A"):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=False)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1, option=option)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2, option=option)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2, option=option)
        self.linear = nn.Linear(64, num_classes)

    # self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride, option):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, option))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        pool_size = int(out.size(3))
        out = F.avg_pool2d(out, pool_size)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet32(option="A"):
    return ResNet(ResnetBlock, [5, 5, 5], option=option)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


In [ ]:
outputs = None
inputs = None
model = None
for i, (images, labels) in enumerate(train_loader):
    if i == 0:
        print(f"Shape of tensor: {images.shape}")
        print(f"Datatype of tensor: {images.dtype}")
        print(f"Device tensor is stored on: {images.device}")

        model = resnet32().to(device)
        images = images.to(device)
        inputs = images
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

In [ ]:
outputs.shape

torch.Size([100, 10])

In [ ]:

inputs.shape

torch.Size([100, 3, 32, 32])

In [ ]:
outputs.data.shape

torch.Size([100, 10])

# **Training**

In [3]:
ts_transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])
ts = torchvision.datasets.CIFAR10(root='../../data/', transform=ts_transform, train=True, download=True)
ts_mean = ts.data.mean(axis=(0,1,2))/255
ts_std = ts.data.std(axis=(0,1,2))/255

print(ts_mean)
print(ts_std)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [4]:
checkpoint = None
try:
    list_of_files = glob.glob(gdrive_location + 'Checkpoint/*') # * means all if need specific format then *.csv
    checkpt = max(list_of_files, key=os.path.getctime)
    checkpoint = torch.load(checkpt)
except:
    pass

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    transforms.Normalize(mean=ts_mean, std=ts_std)])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=True, 
                                            transform=transform,
                                            download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100, 
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 350
learning_rate = 0.1
momentum_val = 0.9
wd=0.0005

#Current epoch
curr_epoch = 0
if checkpoint:
    curr_epoch = checkpoint['epoch']

#Model
model = resnet32().to(device)
if checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = momentum_val, weight_decay = wd)
if checkpoint:
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Train loss
training_loss_array = []
if checkpoint:
    training_loss_ndarray = loadtxt(gdrive_location + 'training_loss_array_'+ str(curr_epoch) +'.csv', delimiter=',')
    training_loss_array =  [training_loss_ndarray.tolist()] if training_loss_ndarray.size == 1 else training_loss_ndarray.tolist()

# Test loss
test_loss_array = []
if checkpoint:
    test_loss_ndarray = loadtxt(gdrive_location + 'test_loss_array_'+ str(curr_epoch) +'.csv', delimiter=',')
    test_loss_array = [test_loss_ndarray.tolist()] if test_loss_ndarray.size == 1 else test_loss_ndarray.tolist()

# Test accuracy
test_accuracy_array = []
if checkpoint:
    test_accuracy_ndarray = loadtxt(gdrive_location + 'test_accuracy_array_'+ str(curr_epoch) +'.csv', delimiter=',')
    test_accuracy_array = [test_accuracy_ndarray.tolist()] if test_accuracy_ndarray.size == 1 else test_accuracy_ndarray.tolist()

#Train accuracy
train_accuracy_array = []
if checkpoint:
    train_accuracy_ndarray = loadtxt(gdrive_location + 'train_accuracy_array_'+ str(curr_epoch) +'.csv', delimiter=',')
    train_accuracy_array = [train_accuracy_ndarray.tolist()] if train_accuracy_ndarray.size == 1 else train_accuracy_ndarray.tolist()

#Scheduler
M1 = 149
M2 = 249
scheduler = MultiStepLR(optimizer, milestones=[M1,M2], gamma=0.1)
if checkpoint:
    scheduler_state_dict = checkpoint['scheduler_state_dict']
    scheduler.load_state_dict(scheduler_state_dict)

# Train the model
total_step = len(train_loader)

for epoch in range(curr_epoch, num_epochs):
    acc_loss = 0.
    n_batches = len(train_loader)
    train_total = 0
    train_correct = 0
    for i, (images, labels) in enumerate(train_loader):

        #Shape of images: torch.Size([100, 3, 32, 32])
        # 100 = # of images in one batch
        # 3 = # of channels
        # 32,32 dimension in pixel

        # Shape of output: torch.Size([100, 10])
        # 100 = # of images in one batch
        # 10 = # of classes
        
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        acc_loss = acc_loss + loss.item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()

        #Recupero dei gradienti?
        # for n, p in model.named_parameters():
        #     print('\n\n' + n)
        #     print(p)

        optimizer.step()

        #train accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} LR: {:.4f}"
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item(), get_lr(optimizer)))
            
    train_accuracy = 100 * train_correct / train_total
    train_accuracy_array.append(train_accuracy)
    print("Train accuracy: {}".format(train_accuracy))

    # Loss of an entire epoch is the average of all batch loss        
    epoch_loss = acc_loss / n_batches
    training_loss_array.append(epoch_loss)     

    scheduler.step()

    # Test the model

    test_accuracy = 0
    confusion_matrix = np.zeros([10,10], int)
    with torch.no_grad():
        test_correct = 0
        test_total = 0
        acc_test_loss = 0.
        n_test_batches = len(test_loader)
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            #torch.max(outputs.data, 1) cerca il massimo rispetto alle righe di output.data
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            # Confusion matrix
            for i, l in enumerate(labels):
                confusion_matrix[l.item(), predicted[i].item()] += 1
            #Test loss
            test_loss = criterion(outputs, labels)
            acc_test_loss = acc_test_loss + test_loss

        epoch_test_loss = acc_test_loss / n_test_batches
        test_loss_array.append(epoch_test_loss)  

        #Test accuracy
        test_accuracy = 100 * test_correct / test_total
        test_accuracy_array.append(test_accuracy)

    print("Test accuracy: {}".format(test_accuracy))

    #Confusion matrix
    classes = ['plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']
    cmn = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=classes, yticklabels=classes)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix epoch ' + str(epoch+1))
    plt.savefig( gdrive_location + 'Confusion_Matrix/confusion_matrix' + str(epoch+1) + '.png')
    plt.clf()
    plt.close(fig)

    #Checkpoint every epochs
    filename = gdrive_location + 'Checkpoint/checkpoint_' + str(epoch+1) + '.tar'
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        'scheduler_state_dict': scheduler.state_dict()
        }, filename)
    #files.download(filename)

    #Loss learning curve
    i = np.arange(1, len(training_loss_array)+1)
    plt.plot(i,training_loss_array, 'purple', label = 'Train loss')
    plt.plot(i,test_loss_array,'green', label = 'Test loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title("Loss curve training set epoch " + str(epoch+1))
    plt.legend(loc = 'best')
    plt.savefig( gdrive_location + 'Loss_Plot/loss_plot' + str(epoch+1) + '.png')
    plt.clf()


    plt.plot(i,train_accuracy_array, 'yellow', label = 'Train accuracy')
    plt.plot(i,test_accuracy_array,'blue', label = 'Test accuracy ')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title("Accuracy curve training set epoch " + str(epoch+1))
    plt.legend(loc = 'best')
    plt.savefig( gdrive_location + 'Accuracy_Plot/accuracy_plot' + str(epoch+1) + '.png')
    plt.clf()

    #save train loss array in csv file
    data1 = asarray([training_loss_array])
    filename_loss1 = gdrive_location + 'training_loss_array_' + str(epoch+1) + '.csv'
    savetxt(filename_loss1, data1, delimiter=',')

    #save test loss array in csv file
    data2 = asarray([test_loss_array])
    filename_loss2 = gdrive_location + 'test_loss_array_' + str(epoch+1) + '.csv'
    savetxt(filename_loss2, data2, delimiter=',')

    #save train accuracy array in csv file
    data3 = asarray([train_accuracy_array])
    filename_accuracy1 = gdrive_location + 'train_accuracy_array_' + str(epoch+1) + '.csv'
    savetxt(filename_accuracy1, data3, delimiter=',')

    #save test accuracy array in csv file
    data4 = asarray([test_accuracy_array])
    filename_accuracy2 = gdrive_location + 'test_accuracy_array_' + str(epoch+1) + '.csv'
    savetxt(filename_accuracy2, data4, delimiter=',')


    

Files already downloaded and verified
Epoch [239/350], Step [100/500] Loss: 0.1552 LR: 0.0100
Epoch [239/350], Step [200/500] Loss: 0.0303 LR: 0.0100
Epoch [239/350], Step [300/500] Loss: 0.0738 LR: 0.0100
Epoch [239/350], Step [400/500] Loss: 0.3044 LR: 0.0100
Epoch [239/350], Step [500/500] Loss: 0.0575 LR: 0.0100
Train accuracy: 96.066
Test accuracy: 86.14
Epoch [240/350], Step [100/500] Loss: 0.0738 LR: 0.0100
Epoch [240/350], Step [200/500] Loss: 0.1987 LR: 0.0100
Epoch [240/350], Step [300/500] Loss: 0.1168 LR: 0.0100
Epoch [240/350], Step [400/500] Loss: 0.0632 LR: 0.0100
Epoch [240/350], Step [500/500] Loss: 0.0768 LR: 0.0100
Train accuracy: 96.062
Test accuracy: 84.64
Epoch [241/350], Step [100/500] Loss: 0.0603 LR: 0.0100
Epoch [241/350], Step [200/500] Loss: 0.0490 LR: 0.0100
Epoch [241/350], Step [300/500] Loss: 0.1193 LR: 0.0100
Epoch [241/350], Step [400/500] Loss: 0.2270 LR: 0.0100
Epoch [241/350], Step [500/500] Loss: 0.0818 LR: 0.0100
Train accuracy: 96.32
Test accurac

<Figure size 432x288 with 0 Axes>

In [10]:
print(epoch_loss)

0.007359617263777181


In [ ]:
list_of_files = glob.glob(gdrive_location + 'Checkpoint/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

/content/drive/MyDrive/TESI/Saved_Data/Checkpoint/checkpoint_3.tar


In [ ]:
checkpoint = None
try:
    checkpoint = torch.load(checkpt)
except:
    pass

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=True, 
                                            transform=transform,
                                            download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=100, 
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/


500

In [ ]:
gradients = {}
# for n, p in model.named_parameters():
#     print()

print(model.conv1.weight.grad)

# for n in model.state_dict().keys():
#     print(n)



None


In [ ]:
gradients = {}
for n, p in model.named_parameters():
    print(layer1.0.bn1.weight.grad)

model.bn1[0].

# for n in model.state_dict().keys():
#     print(n)



SyntaxError: ignored

Learning curve

In [ ]:
# Print learning curve epoch/loss
import numpy as np
import matplotlib.pyplot as plt

i = np.arange(1, len(loss_array)+1)
plt.plot(i,loss_array,'purple')
plt.xlabel('epoch')
plt.ylabel('loss')
#plt.title("")
plt.show()

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        #torch.max(outputs.data, 1) cerca il massimo rispetto alle righe di output.data
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))



Accuracy of the model on the test images: 10.0 %


In [ ]:
optim.zero_grad()
for i in train_loader():
    ...
    loss.backward()
    ...#senza azzerare i gradienti
S = {}
S_norm = {}
for n, p in model.named_parameters():
    S[n] = copy.deepcopy(torch.abs(p.data))
    S_norm[n] = S[n] / torch.max(S[n])
#in S tutte le sensitivity, in un dizionario ordinato layer by layer, in S_norm quelle normalizzate

[1.9587211329936982, 1.3981865257024766, 1.0770300221443176]
